In [239]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from caimcaim import CAIMD # https://github.com/airysen/caimcaim 
# not working for me.
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import StratifiedShuffleSplit


# Display options

%matplotlib
%matplotlib inline
#pd.options.mode.chained_assignment = None #set it to None to remove SettingWithCopyWarning
pd.options.display.float_format = '{:.4f}'.format #set it to convert scientific noations such as 4.225108e+11 to 422510842796.00
pd.set_option('display.max_columns', 100) #  display all the columns
pd.set_option('display.max_rows', 100) # display all the rows
np.set_printoptions(suppress=True,formatter={'float_kind':'{:f}'.format})

Using matplotlib backend: MacOSX


In [ ]:
# functions to be used later in the code.

def impute_numerical_data(num_df):
    """
    Impute all nan values in a numerical
    columns with the median of a correspondingx
    column.
    
    """
    NA_col = num_df.isnull().sum()
    NA_col = NA_col[NA_col>0].index
    
    for column in num_df[NA_col].columns:
        num_df[column].fillna(num_df[column].median(),inplace=True)

    return num_df


def outlier_replacer(df):
    """
    Replace all the outliers in a numerical df using 
    IQR methodology.
    
    """
    for column in df.columns:
        quartile_1, quartile_3 = np.percentile(df[column], [25, 75])
        iqr = quartile_3 - quartile_1
        lower_bound = quartile_1 - (float(iqr) * 1.5)
        upper_bound = quartile_3 + (float(iqr) * 1.5)
        df.loc[(df[column]<lower_bound) | (df[column] > upper_bound),[column]] = df[column].median()
        
    return df


def remove_single_unique_values(dataframe):
    
    """
    Drop all the columns that only contain one unique value.
    not optimized for categorical features yet.
    
    """
    
    cols_to_drop = dataframe.nunique()
    cols_to_drop = cols_to_drop.loc[cols_to_drop.values==1].index
    dataframe = dataframe.drop(cols_to_drop,axis=1)
    return dataframe

In [640]:
# loading data, dropping features with single uniqu values
df = pd.read_csv('2nd_clean.csv')
print('df is loaded')
df.drop('Unnamed: 0',axis='columns',inplace=True)

df is loaded


In [641]:
#train test split
y = df[['loan_status']]
X = df.drop('loan_status',axis='columns')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

In [642]:
# dividing data into categorical and numerical parts
ctgrcl_X_train = X_train.select_dtypes(include=['object'])
nmrcl_X_train = X_train.select_dtypes(exclude=['object'])

# Continuous data treatment

In [646]:
# imputation and outlier treatment (median, IQR)
nmrcl_X_train = impute_numerical_data(nmrcl_X_train)
nmrcl_X_train = outlier_replacer(nmrcl_X_train)
nmrcl_X_train = remove_single_unique_values(nmrcl_X_train)
nmrcl_X_train_columns = nmrcl_X_train.columns
X_train_median_values = nmrcl_X_train.median()

In [647]:
# This is a sub-optimal, temporary measure, as I haven't figured out 
# how to do supervised discretization.
discretizer = KBinsDiscretizer(n_bins=12, encode='ordinal', strategy='kmeans')
discretized_X_train = discretizer.fit_transform(nmrcl_X_train)

/Users/babyhandzzz/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (12). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/babyhandzzz/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 7 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/Users/babyhandzzz/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (12). Possibly due to duplicate points in X.
  return_n_iter=True)
/Users/babyhandzzz/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 29 are removed. Consider decreasing the number of bins.
  'decreasing the numb

In [650]:
# Min_max
scaler = preprocessing.MinMaxScaler()
rescaled_discretized_X_train = scaler.fit_transform(np.array(discretized_X_train))

In [652]:
rescaled_discretized_X_train

array([[0.090909, 0.090909, 0.090909, ..., 0.363636, 0.090909, 0.272727],
       [0.909091, 0.909091, 0.909091, ..., 0.363636, 0.090909, 0.181818],
       [0.636364, 0.636364, 0.636364, ..., 0.454545, 0.272727, 0.000000],
       ...,
       [0.272727, 0.272727, 0.272727, ..., 0.363636, 0.000000, 0.272727],
       [0.363636, 0.363636, 0.363636, ..., 0.727273, 0.272727, 0.545455],
       [0.272727, 0.272727, 0.272727, ..., 0.090909, 0.454545, 0.090909]])

# Question 

I want to use StratifiedShuffleSplit for cross-validation, is it a good choice ?

# Cleaning the categorical data

I have a few ideas on how to deal with the huge number of employee titles but there are enough questions related to the numerical observations for now. So nothing is done with the categorical data.

In [653]:
ctgrcl_X_train.head(2)

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,pymnt_plan,purpose,title,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,disbursement_method,debt_settlement_flag
1404520,36 months,C,C5,Asst. Professor,3 years,RENT,Not Verified,Feb-2017,n,credit_card,Credit card refinancing,AL,Jan-2003,w,Dec-2017,Feb-2019,Individual,N,Cash,N
1686362,60 months,D,D1,"Manager, Channel Sales",10+ years,MORTGAGE,Verified,Dec-2013,n,debt_consolidation,Payoff Debts,CA,Nov-1999,f,Jan-2019,Jan-2019,Individual,N,Cash,N


In [248]:
def check_nan_and_categories(df_column):
    """
    Print value_counts and count of nan
    of a categotical feature.
    
    """
    value_counts = df_column.value_counts()
    nans = df_column.isna().value_counts()
    print(value_counts)
    print('*************************')
    print(nans) 

In [654]:
"""
You probably don't need this, just fillna on the entire df.

"""


# emp_title
ctgrcl_X_train.emp_title.fillna('other',inplace=True)

#emp_length
ctgrcl_X_train.emp_length.fillna('other',inplace=True)

# title
ctgrcl_X_train.title.fillna('other',inplace=True)

# last_pymnt_d
ctgrcl_X_train.last_pymnt_d.fillna('other',inplace=True)

# last_credit_pull_d
ctgrcl_X_train.last_credit_pull_d.fillna('other',inplace=True)

In [655]:
emp_lengthdict = {'10+ years':'ten years or more', 
 '2 years':'two years', 
 '< 1 year':'less than a year', 
 '3 years':'three years', 
 'other':'other', 
 '1 year':'one year',
 '5 years':'five years', 
 '4 years':'four years', 
 '6 years':'six years', 
 '7 years':'seven years', 
 '8 years':'eight years', 
 '9 years':'nine years'}

ctgrcl_X_train.emp_length.replace(emp_lengthdict,inplace=True)

In [656]:
ctgrcl_X_train.drop(['issue_d',
                     'earliest_cr_line',
                     'last_pymnt_d',
                     'last_credit_pull_d'],axis='columns',
                   inplace=True)

# Getting dummies

I tried a few different methods but I haven't been able to 
replace employee titles that are less than 1% with 'other', 
can you please suggest something?

In [657]:
ctgrcl_X_train.shape

(1695501, 16)

In [658]:
ctgrcl_X_train.drop(['emp_title','title'],axis='columns',inplace=True)

In [659]:
ctgrcl_dummies = pd.get_dummies(ctgrcl_X_train,drop_first=True)

In [ ]:
##########

# Stacking categorical and numerical dfs


The dfs are almost ready to be stacked,
I wanted to get it out to you earlier, so some minor things are not yet complete.

In [660]:
final_nmrcl_X_train = pd.DataFrame(rescaled_discretized_X_train)
final_nmrcl_X_train.columns = nmrcl_X_train_columns

In [662]:
final_nmrcl_X_train.index = ctgrcl_dummies.index

In [665]:
final_X_train = pd.concat([final_nmrcl_X_train,ctgrcl_dummies], axis=1)

In [666]:
final_X_train

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,inq_last_6mths,open_acc,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_amnt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 60 months,grade_B,...,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y
1404520,0.0909,0.0909,0.0909,0.5455,0.0909,0.3636,0.7273,0.5000,0.7273,0.1818,1.0000,0.4545,0.0000,0.0000,0.1818,0.1818,0.1818,0.0909,0.3636,0.3636,0.0909,0.6364,0.2727,0.0000,0.8182,0.8182,0.1818,0.7273,0.2727,0.0000,0.4545,0.2727,0.1818,0.2727,0.2000,0.0909,0.9091,0.2727,0.2727,0.2727,0.7273,0.1429,1.0000,0.4545,0.2727,0.3636,0.0909,0.2727,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1686362,0.9091,0.9091,0.9091,0.6364,0.8182,0.7273,0.2727,0.5000,0.5455,0.2727,0.9091,0.3636,0.0000,0.0000,0.2727,0.2727,0.2727,0.2727,0.0000,0.8182,0.1818,0.0000,0.8182,0.0000,0.8182,0.6364,0.4545,0.2727,0.2727,1.0000,0.2727,0.0909,0.3636,0.5455,0.3000,0.1818,0.3636,0.3636,0.2727,0.5455,0.5455,0.0000,1.0000,0.7273,0.8182,0.3636,0.0909,0.1818,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1919269,0.6364,0.6364,0.6364,0.0909,0.8182,0.9091,0.1818,0.0000,0.4545,0.2727,0.6364,0.3636,0.0000,0.0000,0.9091,0.9091,1.0000,0.5455,0.6364,0.0909,1.0000,0.0000,0.0909,0.9091,0.5455,0.5455,0.6364,0.6364,0.9091,0.0000,0.8182,0.2727,0.5455,0.4545,0.7000,0.9091,0.0000,0.5455,0.6364,0.4545,0.4545,0.0000,1.0000,0.2727,0.0909,0.4545,0.2727,0.0000,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
749673,0.2727,0.2727,0.2727,0.4545,0.1818,0.4545,0.1818,0.0000,0.3636,0.3636,0.3636,0.2727,0.0000,0.0000,0.3636,0.3636,0.3636,0.0909,0.3636,0.0000,0.4545,0.1818,0.0000,0.7273,0.1818,0.5455,0.4545,0.6364,0.4545,0.0000,0.3636,0.9091,0.3636,0.4545,0.4000,0.3636,0.0000,0.3636,0.4545,0.4545,0.3636,0.1429,1.0000,0.1818,0.0000,0.1818,0.4545,0.0909,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
869079,0.2727,0.2727,0.2727,0.0909,0.2727,0.5455,0.2727,0.5000,0.3636,0.2727,0.9091,0.4545,0.0000,0.0000,0.3636,0.3636,0.3636,0.0909,0.3636,0.0909,0.0909,0.1818,0.0909,0.0909,0.8182,0.5455,0.5455,0.0000,0.0000,0.4000,0.0000,0.0000,0.8182,0.4545,0.4000,0.6364,0.2727,0.3636,0.4545,0.4545,0.3636,0.1429,1.0000,0.8182,0.0909,0.4545,0.1818,0.3636,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

 There are probably more but the 2 things that I haven't done yet 
 are the proper discretizer and I haven't removed  